Câu 1: Viết chương trình dùng ngôn ngữ Python tính tổng các số từ 1 đến 100 dùng kỹ thuật 
lập trình đa luồng.

In [10]:
import threading

total = 0
lock = threading.Lock()

# Hàm tính tổng các số trong khoảng [start, end]
def sum_range(start, end):
    global total
    local_sum = sum(range(start, end + 1))
    with lock:  # Sử dụng lock để đảm bảo tính toàn vẹn dữ liệu
        total += local_sum

# Chia 2 thread, mỗi thread tính một nửa khoảng
t1 = threading.Thread(target=sum_range, args=(1, 50))
t2 = threading.Thread(target=sum_range, args=(51, 100))

t1.start()  # Bắt đầu thread 1
t2.start()  # Bắt đầu thread 2

t1.join()  # Chờ thread 1 hoàn thành
t2.join()  # Chờ thread 2 hoàn thành

print("Tổng từ 1 đến 100 là:", total)

Tổng từ 1 đến 100 là: 5050


Bài 2: Viết chương trình trong Python dùng kỹ thuật lập trình đa luồng với 3 thread để
thực hiện công việc sau:
- Thread 1: đọc dữ liệu từng số trong file “num_file.txt”. Nếu số đọc được là chẵn thì 
chuyển qua thread 2. Nếu số đọc được là lẻ thì chuyển qua thread 3.
- Thread 2: khi nhận được số của thread 1 thì sẽ in ra các số nguyên tố từ 1 đến số đó.
- Thread 3: khi nhận được số của thread 1 thì sẽ in ra các ước số của nó.
Sau mỗi bước ở trên sẽ có thời gian sleep 1s.

In [8]:
import threading
import time
import queue

even_queue = queue.Queue()  # Hàng đợi cho các số chẵn
odd_queue = queue.Queue()   # Hàng đợi cho các số lẻ

# Hàm kiểm tra số nguyên tố
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5)+1):
        if n % i == 0:
            return False
    return True

# Thread 1: Đọc dữ liệu từ file và phân loại số chẵn/lẻ
def thread_read():
    with open("num.txt") as f:
        for line in f:
            num = int(line.strip())
            if num % 2 == 0:
                even_queue.put(num)  # Đưa số chẵn vào hàng đợi
            else:
                odd_queue.put(num)   # Đưa số lẻ vào hàng đợi
            time.sleep(1)  # Tạm dừng 1 giây

# Thread 2: Xử lý số chẵn, in ra các số nguyên tố từ 1 đến số đó
def thread_even():
    while True:
        num = even_queue.get()
        primes = [i for i in range(1, num + 1) if is_prime(i)]
        print(f"Số nguyên tố từ 1 đến {num}:", primes)
        time.sleep(1)  # Tạm dừng 1 giây

# Thread 3: Xử lý số lẻ, in ra các ước số của nó
def thread_odd():
    while True:
        num = odd_queue.get()
        divisors = [i for i in range(1, num + 1) if num % i == 0]
        print(f"Các ước số của {num}:", divisors)
        time.sleep(1)  # Tạm dừng 1 giây

# Tạo và khởi chạy các thread
t1 = threading.Thread(target=thread_read)
t2 = threading.Thread(target=thread_even, daemon=True)
t3 = threading.Thread(target=thread_odd, daemon=True)

t2.start()  # Bắt đầu thread xử lý số chẵn
t3.start()  # Bắt đầu thread xử lý số lẻ
t1.start()  # Bắt đầu thread đọc dữ liệu

t1.join()  # Chờ thread đọc dữ liệu hoàn thành

Số nguyên tố từ 1 đến 10: [2, 3, 5, 7]
Các ước số của 15: [1, 3, 5, 15]
Các ước số của 15: [1, 3, 5, 15]
Số nguyên tố từ 1 đến 8: [2, 3, 5, 7]
Số nguyên tố từ 1 đến 8: [2, 3, 5, 7]
Các ước số của 7: [1, 7]
Các ước số của 7: [1, 7]
Số nguyên tố từ 1 đến 20: [2, 3, 5, 7, 11, 13, 17, 19]
Số nguyên tố từ 1 đến 20: [2, 3, 5, 7, 11, 13, 17, 19]
Các ước số của 11: [1, 11]
Các ước số của 11: [1, 11]
Các ước số của 13: [1, 13]
Các ước số của 13: [1, 13]
Các ước số của 9: [1, 3, 9]
Các ước số của 9: [1, 3, 9]


Bài 3: Viết một chương trình dùng kỹ thuật lập trình đa luồng trong Python mô phỏng việc 
cập nhật số lượng sản phẩm trong kho. Biết rằng số lượng sản phẩm sẽ bị thay đổi khi có 
một trong hai giao dịch bán hàng hoặc trả hàng

In [9]:
import threading

# Biến toàn cục lưu trữ số lượng sản phẩm
inventory = 100
lock = threading.Lock()

# Hàm mô phỏng giao dịch bán hàng
def sell(quantity):
    global inventory
    with lock:  # Đảm bảo chỉ một thread thay đổi inventory tại một thời điểm
        if inventory >= quantity:
            inventory -= quantity
            print(f"Bán {quantity} sản phẩm. Số lượng còn lại: {inventory}")
        else:
            print("Không đủ hàng để bán.")

# Hàm mô phỏng giao dịch trả hàng
def restock(quantity):
    global inventory
    with lock:
        inventory += quantity
        print(f"Nhập lại {quantity} sản phẩm. Số lượng hiện tại: {inventory}")

# Tạo các thread mô phỏng giao dịch
t1 = threading.Thread(target=sell, args=(30,))
t2 = threading.Thread(target=restock, args=(50,))
t3 = threading.Thread(target=sell, args=(20,))

t1.start()  # Bắt đầu thread bán hàng
t2.start()  # Bắt đầu thread nhập hàng
t3.start()  # Bắt đầu thread bán hàng

t1.join()  # Chờ thread bán hàng hoàn thành
t2.join()  # Chờ thread nhập hàng hoàn thành
t3.join()  # Chờ thread bán hàng hoàn thành

Bán 30 sản phẩm. Số lượng còn lại: 70
Nhập lại 50 sản phẩm. Số lượng hiện tại: 120
Bán 20 sản phẩm. Số lượng còn lại: 100
